In [1]:
using Nemo, Coleman


Welcome to Nemo version 0.17.5

Nemo comes with absolutely no warranty whatsoever


Welcome to 


$\require{action}$


    _    _           _
   | |  | |         | |
   | |__| | ___  ___| | _____
   |  __  |/ _ \/ __| |/ / _ \
   | |  | |  __/ (__|   <  __/
   |_|  |_|\___|\___|_|\_\___|
    
Version 0.8.2 ... 
 ... which comes with absolutely no warranty whatsoever
(c) 2015-2020 by Claus Fieker, Tommy Hofmann and Carlo Sircana



In [2]:
function test_linear_recurrences_modx()
  print("linear recurrences mod x...")
  N = 3
  p = 7
  L = [11,13]
  R = [13,15]

  # Real base ring
  BrR= PadicField(p, N)#ResidueRing(ZZ,Int(p^N))
  RiR,xR = PolynomialRing(BrR,'x')  
  FakeP,k = PolynomialRing(QQ, 'k')
  BrF= FmpqAbsSeriesRing(4,:k)
  k = gen(BrF)
  RiF,xF = PolynomialRing(BrF,'x')  
  MF = matrix(RiF,2,2,[xF + k, xF, 1 + 3*k, xF + 1])
  MR = matrix(RiR,2,2,[xR + p, xR, 1 + 3*p, xR + 1])
  O = Coleman.LinearRecurrence(MF, L, R)
  println(O)

  o = [matrix(BrR,
              [ xadic_to_padic(t[i, j],BrR) for i = 1:nrows(t),j = 1:ncols(t)])
       for t in O]

  o == Coleman.LinearRecurrence(MR, L, R)
  println("PASS")
end

test_linear_recurrences_modx (generic function with 1 method)

In [3]:
test_linear_recurrences_modx()

linear recurrences mod x...AbstractAlgebra.Generic.MatSpaceElem{fmpq_abs_series}[[   168+(61)*k+k^2+O(k^4)  324+(13)*k+O(k^4)]
[26+(79)*k+(3)*k^2+O(k^4)  195+(39)*k+O(k^4)], [   224+(71)*k+k^2+O(k^4)  434+(15)*k+O(k^4)]
[30+(91)*k+(3)*k^2+O(k^4)  255+(45)*k+O(k^4)]]
PASS


Let's do an example of average time Coleman, on an elliptic curve

In [4]:
R,x = PolynomialRing(QQ,"x")
RR,P = PolynomialRing(QQ, 'P')
N = 2
a = ZZ(-16)
b = ZZ(16)
Q = R([b, a, 0, 1])

x^3-16*x+16

What's the answer we are looking for?

In Harvey's version of Kedlaya matrices as follows are used

In [5]:
Delta = 27*b^2+4*a^3

-9472

In [6]:
M_V(t) = QQ(Delta)^(-1) * matrix(QQ,
[[9*b*(6*t-5)   2*a^2*(6*t-5) ]
 [-6*a*(6*t-7)  9*b*(6*t-7)   ]])

M_V (generic function with 1 method)

In [7]:
M_V(ZZ(1))

[-9//592  -2//37]
[ 3//296  9//592]

In [8]:
M_H(t,s) = matrix(QQ,
[[0          0          2*b*s         ]
 [6*t-2*s-3  0          a*(2*s-2*t+1) ]
 [0          6*t-2*s-3  0             ]])

M_H (generic function with 1 method)

In [9]:
M_H(1,1)

[0  0   32]
[1  0  -16]
[0  1    0]

And denominators

In [10]:
D_V(t)    = 2*t-1

D_V (generic function with 1 method)

In [11]:
D_H(t, s) = 6*t-2*s-3

D_H (generic function with 1 method)

We will use only two digits of precision for now, and we are in genus 1

In [12]:
N,g = 2,1

(2, 1)

In [13]:
C(j, r) = coeff(Q^j, r)

C (generic function with 1 method)

In [14]:
# B(j, r) = p*C(j, r)*sum([(-1)^(k+j)*prod([-QQ(1)//2 - i for i in 0:(k-1)])/(factorial(k)) for k in j..(N-1)])

In [15]:
function B(j, r)  # Stole this from my Coleman repo
    lambda = coeff(Q^j, r)
    # num = numerator of the two binomial expressions
    num = FlintQQ(1)
    for i = 0:N-2
        num = num*(-(1//2)-i)
    end
    # denom = denominator of the two binomial expressions
    denom = factorial(j)*factorial((N-1)-j)
    # last summand
    summand = num//denom
    sum = (-1)^(N-1+j)*summand
    # summing up going down
    for l = (N-2):-1:j
        summand = summand*(l+1-j)//(-(1//2)-l)
        sum = sum + (-1)^(l+j)*summand
    end
    P*lambda*sum
end

B (generic function with 1 method)

In [16]:
B(1,1)

8*P

The main formula is
$$\sigma(x^i d x / y) \equiv T_{i}=\sum_{j=0}^{N-1} \sum_{r=0}^{(2 g+1) j} B_{j, r} x^{p(i+r+1)-1} y^{-p(2 j+1)+1} d x / y \pmod {p^N}$$

$$ N = 2 $$
$$ g = 1 $$
$$ T_0 =  B(0, 0)x^{p-1} y^{-p + 1} dx/y + y^{-3p + 1}\left(B(1,0) x^{p-1} +  B(1,1) x^{2p - 1} + B(1, 2) x^{3p - 1}  +  B(1, 3)x^{4p - 1} \right)dx/y$$
lying in
$$W_{p - 1, (-p+1)/2},W_{p - 1, (-3p+1)/2},W_{2p - 1, (-3p+1)/2},W_{3p - 1, (-3p+1)/2},W_{4p - 1, (-3p+1)/2}$$
respectively, represented as $(0,0,\ldots,B(?,?))$
so we need to use
$$M_{H}^{(-p+1)/2}(p-1),M_{H}^{(-3p+1)/2}(p-1),M_{H}^{(-3p+1)/2}(2p-1),M_{H}^{(-3p+1)/2}(3p-1),M_{H}^{(-3p+1)/2}(4p-1)$$

In [38]:
M_H(t,s1,s2,T) = prod(s->T(M_H(t,s)),s1+1:s2)
M_V(t1,t2,T) = prod(t->T(M_V(t)),t1+1:t2)
D_H(t,s1,s2,T) = prod(s->T(D_H(t,s)),s1+1:s2)
D_V(t1,t2,T) = prod(t->T(D_V(t)),t1+1:t2)

D_V (generic function with 3 methods)

In [39]:
M_H(1,1,3) == M_H(1,2)*M_H(1,3)

true

In [40]:
M_V(1,3) == M_V(2)*M_V(3)

true

In [41]:
[i for i in (-4:-5)]

0-element Array{Int64,1}

In [64]:
N = 2
p = 11
for i = 0:1
    curV = matrix(RR, [fmpq_poly[zero(RR)] for I in 1:2*g])
    for j in N-1:-1:0 # Vertical
        curH = matrix(RR, [fmpq_poly[zero(RR)] for I in 1:2*g+1])
        for r in (2*g + 1)*j:-1:0 # Horizontal
            curH[2*g+1,1] += B(j, r) # x^(p*(i+r+1)-1) y^(-p*(2*j+1)+1)
            T = MatrixSpace(QQ, 2*g + 1, 2*g + 1)
            print(T)
            curH = RR(1//D_H((p*(2*j+1)-1)//2,p*(i+r)-1,p*(i+r+1)-1,QQ))*change_base_ring(RR, M_H((p*(2*j+1)-1)//2,p*(i+r)-1,p*(i+r+1)-1,T)) * curH
            println(j, " ", r, " ", coeff)
        end
        for I in 1:2*g
            curV[I,1] += curH[I+1,1]
        end
        T = MatrixSpace(QQ, 2*g + 1, 2*g + 1)
        curV = RR(1//D_V((p*(2*(j-1)+1)-1)//2,(p*(2*j+1)-1)//2, QQ)) * change_base_ring(RR, M_V((p*(2*(j-1)+1)-1)//2,(p*(2*j+1)-1)//2,T)) * curV
        println(curV)
    end
    K = PadicField(p, N)
    println(map(P->K(P(p)), curV))
end

Matrix Space of 3 rows and 3 columns over Rational Field1 3 coeff
Matrix Space of 3 rows and 3 columns over Rational Field1 2 coeff
Matrix Space of 3 rows and 3 columns over Rational Field1 1 coeff
Matrix Space of 3 rows and 3 columns over Rational Field1 0 coeff
[2028908651887211469336218978619039//653131454348333548677010554880*P]
[-212065116244739331986510433581317//161117308367095543838095441920*P]
Matrix Space of 3 rows and 3 columns over Rational Field0 0 coeff
[-10049419841526204316854647478092255//13839745426291611458774353564942872292098048*P]
[ -3494236540577274725753189215284013//10764246442671253356824497217177789560520704*P]
[0 + O(11^2)]
[0 + O(11^2)]
Matrix Space of 3 rows and 3 columns over Rational Field1 3 coeff
Matrix Space of 3 rows and 3 columns over Rational Field1 2 coeff
Matrix Space of 3 rows and 3 columns over Rational Field1 1 coeff
Matrix Space of 3 rows and 3 columns over Rational Field1 0 coeff
[         4026234044698382612420082164778853465753839409644887

In [65]:
R, x = PolynomialRing(Nemo.GF(11), "x")
h = x^3 + a*x + b
ZetaFunction(2, h)

(11*t^2+5*t+1)//(11*t^2-12*t+1)


signal (15): Terminated: 15
in expression starting at /Users/alex/.julia/packages/IJulia/DrVMH/src/kernel.jl:52


In [63]:
Rp = PolynomialRing(FlintPadicField(p, N), "t")[1]
Qp = Rp([b, a, 0, 1])
AbsoluteFrobeniusActionOnLift(2, Qp, 2, p, 1)

[9*11^1 + 9*11^2 + O(11^3)  2*11^1 + 2*11^2 + O(11^3)]
[    10 + 1*11^1 + O(11^2)       6 + 1*11^1 + O(11^2)]

In [24]:
sage_run(s) = read(`sage -c "$s"`, String)
sage_run("
load('hgm.py')
print('hello')
print('ayy')
")

Traceback (most recent call last):
  File "/Users/alex/sage/src/bin/sage-eval", line 4, in <module>
    from sage.all import *
  File "/Users/alex/sage/local/lib/python3.7/site-packages/sage/all.py", line 119, in <module>
    from sage.misc.all       import *         # takes a while
  File "/Users/alex/sage/local/lib/python3.7/site-packages/sage/misc/all.py", line 84, in <module>
    from .functional import (additive_order,
  File "/Users/alex/sage/local/lib/python3.7/site-packages/sage/misc/functional.py", line 26, in <module>
    from sage.rings.complex_double import CDF
ImportError: dlopen(/Users/alex/sage/local/lib/python3.7/site-packages/sage/rings/complex_double.cpython-37m-darwin.so, 2): Library not loaded: /usr/local/opt/gcc/lib/gcc/9/libgfortran.5.dylib
  Referenced from: /Users/alex/sage/local/lib/libopenblas_haswellp-r0.3.9.dylib
  Reason: image not found


ProcessFailedException: failed process: Process(`sage -c "
load('hgm.py')
print('hello')
print('ayy')
"`, ProcessExited(1)) [1]
